In [1]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

def visualiseCorr(corr):
    fig, ax = plt.subplots(figsize=(40,15))
    image = corr
    ax.imshow(image, cmap=plt.cm.gray, interpolation='nearest')
    ax.set_title('dropped spines')
    
    # Move left and bottom spines outward by 10 points
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    # Only show ticks on the left and bottom spines
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    plt.show()
    
def threshhold(corr, thld):
    w = len(corr[0])
    h = len(corr)
    tcorr = np.zeros((h, w))
    for i_index, i in enumerate(corr):
        for j_index, j in enumerate(i):
            if j < thld:
                tcorr[i_index][j_index] = 0
            else:
                tcorr[i_index][j_index] = j
    return tcorr
    
def postprocess(corr, windowsize):
    w = len(corr[0])
    h = len(corr)
    #print (w,h)
    M = 4
    ncorr = np.zeros((M*h, 2*w+windowsize))
    for i,j in enumerate(corr):
        #print (len(ncorr[i][-w:]), len(j))
        for k in range(M):
            ncorr[i*M+k][-w-((1+i)*windowsize):-(1+i)*windowsize] = j
        #print(len(ncorr[i][-w-((1+i)*windowsize):-(1+i)*windowsize]))
    
    return(ncorr)

In [2]:
from ipyparallel import Client
import numpy as np
c = Client(profile='default')
dview = c[:]

@dview.parallel(block=True)
def correlateblock(data):
    import numpy as np
    res = []
    for d in data:
        a = d['a']
        b = d['b']
        block = d['block']
        #if not block % 5:
        #print(block)
        windowsize = d['windowsize']
        corr = np.zeros(len(a)-windowsize)
        y = b[block*windowsize:(1+block)*windowsize]
        for j in range(len(a)-windowsize):
            x = a[0+j:windowsize+j]
            corr[j] = np.corrcoef(x, y)[0,1]
            #if not j % 5:
            #    print(j, end='')
        res.append({'block':block, 'corr':corr})
    return res
    

def correlate(a, b, windowsize): #, dump="dump", start=0, stop=0):
    corr = np.zeros((int(len(b)/windowsize), len(a)-windowsize))
    totalJobs = int(len(b)/windowsize) * len(a)-windowsize
    
    
    jobs = [{'a':a, 'b':b, 'block':i, 'windowsize':windowsize} for i in range(0, int(len(b)/windowsize))]    
    print("total jobs:", len(jobs))#print([i for i in range(0, int(len(b)/windowsize))])
    chunk = 4
    for i in range(int(len(jobs)/chunk)):
        print(4*i,4*(i+1))
        data = correlateblock(jobs[4*i:4*(i+1)])
        for i in data:
            corr[i['block']] = i['corr']
    return corr

In [3]:
length = 1000
windowsize = 40
a = np.random.uniform(size=(length))
b = np.random.uniform(size=(length))
b[-500:] = a[-500:]
blocks = int(len(b)/windowsize)

In [4]:
corr = correlate(a, a, windowsize)

total jobs: 25
0 4
4 8
8 12
12 16
16 20
20 24


In [5]:
visualiseCorr(corr)
tcorr = threshhold(corr, 0.005)
visualiseCorr(tcorr)
ncorr = postprocess(tcorr, windowsize)
visualiseCorr(ncorr)

In [6]:
master = np.genfromtxt(
    'master.csv',           # file name
    skip_header=5,          # lines to skip at the top
    skip_footer=0,          # lines to skip at the bottom
    delimiter=',',          # column delimiter
    dtype='float64',        # data type
    filling_values=0,       # fill missing values with 0
    usecols = (0,1),    # columns to read
    names=['Time', ',Ampl'])     # column names
slave = np.genfromtxt(
    'slave.csv',           # file name
    skip_header=5,          # lines to skip at the top
    skip_footer=0,          # lines to skip at the bottom
    delimiter=',',          # column delimiter
    dtype='float32',        # data type
    filling_values=0,       # fill missing values with 0
    usecols = (0,1),    # columns to read
    names=['Time', 'Ampl'])     # column names

In [ ]:
a = master['Ampl']
b = slave['Ampl']
print(len(a), len(b))

400002 400002


In [ ]:
windowsize = 1000
corr = correlate(a[1000:], b[1000:], windowsize)

total jobs: 399
0 4
